In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('data_Q3_2019/2019-09-30.csv')

In [2]:
df.head()

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2019-09-30,Z305B2QN,ST4000DM000,4000787030016,0,115.0,88847920.0,NaN,NaN,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-09-30,ZJV0XJQ4,ST12000NM0007,12000138625024,0,77.0,53191496.0,NaN,NaN,96.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-09-30,ZJV0XJQ3,ST12000NM0007,12000138625024,0,80.0,107432696.0,NaN,NaN,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-09-30,ZJV0XJQ0,ST12000NM0007,12000138625024,0,83.0,198398656.0,NaN,NaN,93.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-09-30,PL1331LAHG1S4H,HGST HMS5C4040ALE640,4000787030016,0,100.0,0.0,134.0,103.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Verify all_df
df[['serial_number','smart_9_raw']][df.serial_number=='S30116CX'].max()

serial_number    S30116CX
smart_9_raw         38943
dtype: object

In [4]:
df.shape

(115150, 129)

In [5]:
df.model.unique()

array(['ST4000DM000', 'ST12000NM0007', 'HGST HMS5C4040ALE640',
       'ST8000NM0055', 'ST8000DM002', 'HGST HMS5C4040BLE640',
       'HGST HUH721212ALN604', 'TOSHIBA MG07ACA14TA',
       'HGST HUH721212ALE600', 'TOSHIBA MQ01ABF050', 'ST500LM030',
       'ST6000DX000', 'ST10000NM0086', 'DELLBOSS VD',
       'TOSHIBA MQ01ABF050M', 'WDC WD5000LPVX', 'ST500LM012 HN',
       'HGST HUH728080ALE600', 'TOSHIBA MD04ABA400V', 'TOSHIBA HDWF180',
       'ST8000DM005', 'Seagate SSD', 'HGST HUH721010ALE600',
       'ST4000DM005', 'WDC WD5000LPCX', 'HGST HDS5C4040ALE630',
       'ST500LM021', 'Hitachi HDS5C4040ALE630', 'HGST HUS726040ALE610',
       'Seagate BarraCuda SSD ZA500CM10002', 'ST12000NM0117',
       'Seagate BarraCuda SSD ZA2000CM10002',
       'Seagate BarraCuda SSD ZA250CM10002', 'TOSHIBA HDWE160',
       'WDC WD5000BPKT', 'ST6000DM001', 'WDC WD60EFRX', 'ST8000DM004',
       'HGST HMS5C4040BLE641', 'ST1000LM024 HN', 'ST6000DM004'],
      dtype=object)

In [6]:
df.capacity_bytes.sort_values().unique()

array([            -1,   250059350016,   480036847616,   500107862016,
        1000204886016,  2000398934016,  4000787030016,  6001175126016,
        8001563222016, 10000831348736, 12000138625024, 14000519643136])

In [7]:
df.capacity_bytes.describe()

count    1.151500e+05
mean     8.440954e+12
std      3.610303e+12
min     -1.000000e+00
25%      4.000787e+12
50%      8.001563e+12
75%      1.200014e+13
max      1.400052e+13
Name: capacity_bytes, dtype: float64

In [8]:
def get_manufacturer(model):
    if model.startswith('ST'):
        return 'seagate'
    elif model.startswith('Sea'):
        return 'seagate'
    elif model.startswith('HGST'):
        return 'hitachi'
    elif model.startswith('Hit'):
        return 'hitachi'
    elif model.startswith('TOS'):
        return 'toshiba'
    elif model.startswith('DEL'):
        return 'dell'
    elif model.startswith('WD'):
        return 'western digital'
    elif model.startswith('Sam'):
        return 'samsung'
    elif model.startswith('SAM'):
        return 'samsung'
    else:
        return 'unknown'

In [9]:
df['manufacturer'] = df.model.apply(get_manufacturer)
df[['model', 'manufacturer']].head()

,model,manufacturer
0,ST4000DM000,seagate
1,ST12000NM0007,seagate
2,ST12000NM0007,seagate
3,ST12000NM0007,seagate
4,HGST HMS5C4040ALE640,hitachi


In [10]:
df.manufacturer.isnull().sum()

0

In [11]:
df.manufacturer.value_counts()

seagate            83768
hitachi            28834
toshiba             2207
western digital      281
dell                  60
Name: manufacturer, dtype: int64

In [12]:
# list comp. to find index of a value in a series/column
# df['manufacturer'].loc[lambda x: x=='unknown'].index

In [13]:
#df.iloc[5913]

- The following cells are for my reference they are misc. commands

In [14]:
# to view all columns instead of ......
# cols = df.columns.tolist()
# cols

# to view stats on SMART STAT
# df.smart_9_raw.describe()

# to view nulls because overall count is different
# df.smart_9_raw.isnull().sum()

# to view all non-null columns in a record
# df.iloc[54718].dropna()

# to view 2 columns for a specific model and its SMART STAT value
# df[['model','smart_9_raw']][df.model=='ST12000NM0007']

# David's function
'''
def get_manufacturer(df):
    models = df['model'].str[0:-1]
    manufacturers = []
    for i in models:
        if i[0:2] == 'ST':
            manufacturers.append('Seagate')
        else:
            manufacturers.append('Unknown')
    df['manufacturer'] = manufacturers
    return df  
'''

"\ndef get_manufacturer(df):\n    models = df['model'].str[0:-1]\n    manufacturers = []\n    for i in models:\n        if i[0:2] == 'ST':\n            manufacturers.append('Seagate')\n        else:\n            manufacturers.append('Unknown')\n    df['manufacturer'] = manufacturers\n    return df  \n"

### New aggregated data

In [15]:
all_df = pd.read_csv('hard_drives_smart_5.csv')

In [16]:
all_df.head(25)

,Unnamed: 0,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_188_raw),max(smart_197_raw),max(smart_198_raw)
0,0,PL1311LAG1SJAA,Hitachi HDS5C4040ALE630,4000787030016,0,43819.0,0.0,NaN,NaN,0.0,0.0
1,1,Z305KB36,ST4000DM000,4000787030016,0,31045.0,0.0,0.0,0.0,0.0,0.0
2,2,MJ0351YNG9MZXA,Hitachi HDS5C3030ALA630,3000592982016,0,41668.0,0.0,NaN,NaN,0.0,0.0
3,3,ZA11NHSN,ST8000DM002,8001563222016,0,26284.0,0.0,0.0,0.0,0.0,0.0
4,4,MJ1311YNG2ZSEA,Hitachi HDS5C3030ALA630,3000592982016,0,47994.0,0.0,NaN,NaN,0.0,0.0
5,5,Z305D5VF,ST4000DM000,4000787030016,0,31883.0,0.0,0.0,0.0,0.0,0.0
6,6,Z3015L2E,ST4000DM000,4000787030016,0,36768.0,0.0,0.0,0.0,0.0,0.0
7,7,PL1331LAHDYJYH,HGST HMS5C4040BLE640,4000787030016,0,30144.0,0.0,NaN,NaN,0.0,0.0
8,8,Z304LAHP,ST4000DM000,4000787030016,0,33885.0,0.0,0.0,0.0,0.0,0.0
9,9,Z302B0GK,ST4000DM000,4000787030016,0,36443.0,0.0,0.0,0.0,0.0,0.0


In [17]:
all_df.shape

(169073, 11)

In [18]:
all_df.columns

Index(['Unnamed: 0', 'serial_number', 'model', 'capacity_bytes',
       'max(failure)', 'max(smart_9_raw)', 'max(smart_5_raw)',
       'max(smart_187_raw)', 'max(smart_188_raw)', 'max(smart_197_raw)',
       'max(smart_198_raw)'],
      dtype='object')

In [19]:
all_df.dtypes

Unnamed: 0              int64
serial_number          object
model                  object
capacity_bytes          int64
max(failure)            int64
max(smart_9_raw)      float64
max(smart_5_raw)      float64
max(smart_187_raw)    float64
max(smart_188_raw)    float64
max(smart_197_raw)    float64
max(smart_198_raw)    float64
dtype: object

In [20]:
all_df.isna().sum()

Unnamed: 0                0
serial_number             1
model                     0
capacity_bytes            0
max(failure)              0
max(smart_9_raw)       7098
max(smart_5_raw)       7222
max(smart_187_raw)    64884
max(smart_188_raw)    64894
max(smart_197_raw)     7232
max(smart_198_raw)     7232
dtype: int64

In [21]:
all_df.model.nunique()

103

### Run function on inventory list

In [22]:
all_df['manufacturer'] = all_df.model.apply(get_manufacturer)
all_df[['model', 'manufacturer']].head(50)

,model,manufacturer
0,Hitachi HDS5C4040ALE630,hitachi
1,ST4000DM000,seagate
2,Hitachi HDS5C3030ALA630,hitachi
3,ST8000DM002,seagate
4,Hitachi HDS5C3030ALA630,hitachi
5,ST4000DM000,seagate
6,ST4000DM000,seagate
7,HGST HMS5C4040BLE640,hitachi
8,ST4000DM000,seagate
9,ST4000DM000,seagate


In [23]:
all_df.manufacturer.isnull().sum()

0

In [24]:
all_df.manufacturer.value_counts()

seagate            110097
hitachi             53473
western digital      2826
toshiba              2604
dell                   60
samsung                11
unknown                 2
Name: manufacturer, dtype: int64

In [25]:
all_df['manufacturer'].loc[lambda x: x== 'unknown'].index

Int64Index([33381, 53781], dtype='int64')

In [26]:
all_df.iloc[33381]

Unnamed: 0                    33381
serial_number             0F112CC45
model                        00MD00
capacity_bytes        4000787030016
max(failure)                      0
max(smart_9_raw)              14476
max(smart_5_raw)                  0
max(smart_187_raw)                0
max(smart_188_raw)                0
max(smart_197_raw)                0
max(smart_198_raw)                0
manufacturer                unknown
Name: 33381, dtype: object

In [27]:
all_df.iloc[53781]

Unnamed: 0                    53781
serial_number                   NaN
model                        00MD00
capacity_bytes        4000787030016
max(failure)                      0
max(smart_9_raw)              14476
max(smart_5_raw)                  0
max(smart_187_raw)                0
max(smart_188_raw)                1
max(smart_197_raw)                0
max(smart_198_raw)                0
manufacturer                unknown
Name: 53781, dtype: object

In [39]:
all_df.head()

,Unnamed: 0,serial_number,model,capacity_bytes,max(failure),max(smart_9_raw),max(smart_5_raw),max(smart_187_raw),max(smart_188_raw),max(smart_197_raw),max(smart_198_raw),manufacturer
0,0,PL1311LAG1SJAA,Hitachi HDS5C4040ALE630,4000787030016,0,43819.0,0.0,NaN,NaN,0.0,0.0,hitachi
1,1,Z305KB36,ST4000DM000,4000787030016,0,31045.0,0.0,0.0,0.0,0.0,0.0,seagate
2,2,MJ0351YNG9MZXA,Hitachi HDS5C3030ALA630,3000592982016,0,41668.0,0.0,NaN,NaN,0.0,0.0,hitachi
3,3,ZA11NHSN,ST8000DM002,8001563222016,0,26284.0,0.0,0.0,0.0,0.0,0.0,seagate
4,4,MJ1311YNG2ZSEA,Hitachi HDS5C3030ALA630,3000592982016,0,47994.0,0.0,NaN,NaN,0.0,0.0,hitachi


In [29]:
all_df.manufacturer.unique()

array(['hitachi', 'seagate', 'western digital', 'toshiba', 'dell',
       'samsung', 'unknown'], dtype=object)

In [30]:
all_df.model.value_counts()

ST12000NM0007                          39779
ST4000DM000                            36802
ST8000NM0055                           17305
HGST HMS5C4040BLE640                   17011
HGST HUH721212ALN604                   12036
ST8000DM002                            10345
HGST HMS5C4040ALE640                    8813
Hitachi HDS5C3030ALA630                 4563
Hitachi HDS722020ALA330                 4503
Hitachi HDS5C4040ALE630                 2639
ST6000DX000                             1942
HGST HUH721212ALE600                    1564
ST10000NM0086                           1329
TOSHIBA MG07ACA14TA                     1328
WDC WD30EFRX                            1167
HGST HUH728080ALE600                    1102
Hitachi HDS723030ALA640                 1000
ST500LM012 HN                            804
TOSHIBA MQ01ABF050                       589
WDC WD60EFRX                             471
ST4000DM001                              425
TOSHIBA MQ01ABF050M                      410
WDC WD5000

In [31]:
all_df.model.describe()

count            169073
unique              103
top       ST12000NM0007
freq              39779
Name: model, dtype: object

In [32]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169073 entries, 0 to 169072
Data columns (total 12 columns):
Unnamed: 0            169073 non-null int64
serial_number         169072 non-null object
model                 169073 non-null object
capacity_bytes        169073 non-null int64
max(failure)          169073 non-null int64
max(smart_9_raw)      161975 non-null float64
max(smart_5_raw)      161851 non-null float64
max(smart_187_raw)    104189 non-null float64
max(smart_188_raw)    104179 non-null float64
max(smart_197_raw)    161841 non-null float64
max(smart_198_raw)    161841 non-null float64
manufacturer          169073 non-null object
dtypes: float64(6), int64(3), object(3)
memory usage: 15.5+ MB


In [33]:
all_df.manufacturer.value_counts()

seagate            110097
hitachi             53473
western digital      2826
toshiba              2604
dell                   60
samsung                11
unknown                 2
Name: manufacturer, dtype: int64

In [34]:
#check duplicate serial numbers
counts = all_df.serial_number.value_counts() > 1

In [35]:
counts.sum()

7041

In [36]:
(df.serial_number=='PL2331LAHB93UJ')

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
10        False
11        False
12        False
13        False
14        False
15        False
16        False
17        False
18        False
19        False
20        False
21        False
22        False
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
115120    False
115121    False
115122    False
115123    False
115124    False
115125    False
115126    False
115127    False
115128    False
115129    False
115130    False
115131    False
115132    False
115133    False
115134    False
115135    False
115136    False
115137    False
115138    False
115139    False
115140    False
115141    False
115142    False
115143    False
115144    False
115145    False
115146    False
115147    False
115148    False
115149    False
Name: serial_number, Len

In [37]:
df[['serial_number', 'model','smart_9_raw']][df.serial_number=='PL2331LAHB93UJ']

,serial_number,model,smart_9_raw
77265,PL2331LAHB93UJ,HGST HMS5C4040ALE640,5141.0


In [38]:
all_df.serial_number.value_counts()

PL2331LAHB93UJ    3
PL1331LAHD6JBH    2
ZA1818L4          2
ZA189JNC          2
45D7K133FMYB      2
ZA189W0G          2
Z300JZGN          2
ZCH08BXK          2
VKGK9LGX          2
ZA181466          2
Z300GQ4B          2
ZA18CE6C          2
ZCH06DSJ          2
ZA18145V          2
ZA181576          2
ZA14EL0Y          2
ZA17ZNAZ          2
ZCH02KRC          2
ZA13VMWT          2
ZA13G9QQ          2
ZA180QC4          2
ZA12SQN8          2
ZA12Y27D          2
ZA17ZNAL          2
2AGPKTPY          2
ZA1811D2          2
S300X9DN          2
ZA18122K          2
PL1331LAHBGA3H    2
ZA180R3R          2
                 ..
8CH171ME          1
PL1321LAG329HH    1
ZA17Z102          1
ZCH08B9L          1
ZA143SHD          1
PL2331LAHDX7WJ    1
ZA12RAYR          1
JK11A8B9J0DUUF    1
PL1331LAGSKHNH    1
PL2331LAGMNXUJ    1
PL1331LAGM0BTH    1
AAGXU99H          1
ZJV0WEZZ          1
S300Z6X9          1
ZCH084BW          1
ZA1621JG          1
AAG6Z04H          1
JK1101B8KNHL3S    1
Z300JC83          1
